In [ ]:
#get packages we will use
from math import pi
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.models.sources import ColumnDataSource
from bokeh.models import HoverTool
output_notebook()

In [ ]:
#import data and make a normalized copy of it
df = pd.read_excel('HexaPropsAssay.xlsx', index_col=0)
cdtree = pd.read_csv('cdtree.csv')
ndf = (df - df.min(axis=0)) / (df.max(axis=0) - df.min(axis=0))

In [ ]:
#create dendrogram tree via dicitonary
import collections
parents = cdtree['parent'].tolist()
children = cdtree['child'].tolist()
forwardtree = collections.defaultdict(list)
reversetree = {}
for x,y in zip(parents,children):
    reversetree[y] = x
    forwardtree[x].append(y)

In [ ]:
#create dendrogram, each line is a rect glyph using multiline
'''
Based on https://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.multi_line
we can put as many random lines in the space below as we want - so we can use that to make our dendrogram.
It takes a list of lists as its xs/ys, meaning if xs=[[1, 2], [3, 4]] and ys=[[4, 3], [2, 1]], then there are two
lines. The first has two x points (1 and 2) and two y points (4 and 3). The first line is then defined as:
starting from (1, 4) going to (2, 3).

So based on the groupings of the children, we can build the positioning of where each line goes backwards.
The lenth of each line could be of length 1 or 2, the x range would need to corrispond to how long you want it to
be though. Think there's around 900+ nodes to take care of.
'''
#highest parent is row 838, which shall be at the bottom. All parents will take the bottome path while their
#children go up.
xs=[]
ys=[]
name=[]

def recursivebuild(parent, x=.5, y=.5):
    yi = y+1
    if parent in forwardtree:
        bottom = yi
        for child in forwardtree[parent]:
            if child <= 837:
                xs.append([x, 200])
                ys.append([yi, yi])
                name.append(child)
            yi = recursivebuild(child, x+4, yi)
        top = yi
        xs.append([x+4, x+4])
        ys.append([bottom, top])
        name.append('child connector')
        xs.append([x, x+4])
        ys.append([bottom+top/2, bottom+top/2])
        name.append('parent to child connector')
        xs.append([x, x])
        ys.append([y, bottom+top/2])
        name.append('parent extender')
        #make vertical line
    return yi

print(recursivebuild(838))

dendo = figure(x_range = [0,200], y_range = [0,1000])
dendo.multi_line(xs='xs', ys='ys', source=ColumnDataSource(dict(xs=xs, ys=ys, name=name)))
h = HoverTool()
h.tooltips = [("Node #", "@name")]
dendo.add_tools(h)


dendo.grid.grid_line_color = None
dendo.axis.axis_line_color = None
show(dendo)

In [ ]:
#reshape the data to be easily understood by bokeh
molecule = []
attribute = []
xs = []
ys = []
colors = []
alpha = []
value = []

for i, m in enumerate(df.index):
    molecule = molecule + [m]*len(df.columns)
    attribute = attribute + df.columns.tolist()
    xs = xs + list(np.arange(0.5, len(df.columns)+0.5))
    ys = ys + [i+0.5]*len(df.columns)
    value = value + df.loc[m].tolist()
    alpha = alpha + ndf.loc[m].tolist()

data = pd.DataFrame(dict(
        molecule = molecule,
        attribute = attribute, 
        xs = xs,
        ys = ys,
        value = value,
        alpha = alpha
              ))

source = ColumnDataSource(data)

In [ ]:
#create tooltip and heatmap
hover = HoverTool()

hover.tooltips = [("Molecule", "@molecule"), ("Attribute", "@attribute"), ("Value", "@value")]

hm = figure(x_range=df.columns.tolist(), y_range=[0, df.shape[0]])
hm.rect(x='xs', y='ys', height=1, width=1, source=source, fill_alpha='alpha', line_alpha=0.1)

hm.xaxis.major_label_orientation = pi/2
hm.xaxis.major_label_text_font_size = '7pt'
hm.axis.major_tick_line_color = None
hm.axis.minor_tick_line_color = None
hm.yaxis.major_label_text_color = None
hm.grid.grid_line_color = None
hm.axis.axis_line_color = None

hm.add_tools(hover)
show(hm)